# 基于简单 TF-IDF的 标签测试

基本思路就是将问题与问题描述用到的词放在一起，当做一篇文章，然后计算全部问题的IDF,再将标签当做关键词计算标签与问题的相似度。

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time 
import operator
from collections import Counter
from ast import literal_eval
import os
import gc

%matplotlib inline

## Parallelization of pandas.apply() 

根据 https://stackoverflow.com/questions/37078880/status-of-parallelization-of-pandas-apply

找到下面两个并行计算的网址

https://github.com/pandas-dev/pandas/issues/13111

http://www.racketracer.com/2016/07/06/pandas-in-parallel/

我试验了dask 超快，下面有几个框，先用传统的pandas做，然后再用dask做，你可以比较一下，最好能比较一下看看结果是不是一样的。如果 结果一样的话，就可以把传统的实现删除了。




In [2]:
import dask
import dask.dataframe as dd
from dask import threaded, multiprocessing

## Read in data sets

将原来的数据路径放在环境变量里面，这样就不用每次改程序了

nrows: 考虑到曾经在328877那里曾经报过错，所以编程时至少加载35万条数据, 在read_csv时会用到

In [3]:
#data_path=os.environ.get('zhihu_data_path')+'/' 
data_path = 'ieee_zhihu_cup/'
nrows=350 * 1000 

In [4]:
start_time = time.time()
print('Start time:', start_time)
df_questions = pd.read_csv(data_path+'question_train_set.txt',header=None, names=['question_id', 'ct', 'wt','cd','wd'], sep='\t', nrows=nrows)
print('time cost:', time.time() - start_time)

Start time: 1498001393.2119536
time cost: 3.727313756942749


### 将DataFrame 转成dask

In [5]:
start_time = time.time()
ddf_questions = dd.from_pandas(df_questions, npartitions =4)
print('time cost:', time.time() - start_time)

time cost: 1.529158115386963


## Prepare df_questions

In [6]:
def split(row):
    return [] if type(row) == float else row.split(',')

### 两个数据异常
3 是没有description

328877 是有title 里面的字不成词

In [7]:
print (df_questions.loc[3])
print (df_questions.loc[328877])

question_id                                    -5698296155734268
ct             c473,c1528,c528,c428,c295,c15,c101,c188,c146,c...
wt             w8646,w2744,w1462,w9,w54,w138,w54,w50,w110,w14...
cd                                                           NaN
wd                                                           NaN
Name: 3, dtype: object
question_id                                  -930822265097814910
ct             c149,c148,c148,c42,c185,c95,c95,c190,c42,c106,...
wt                                                           NaN
cd             c346,c818,c740,c630,c101,c1128,c386,c740,c630,...
wd                              w61033,w54,w26250,w973,w54,w9892
Name: 328877, dtype: object


In [8]:
df_questions.head(5)

,question_id,ct,wt,cd,wd
0,6555699376639805223,"c324,c39,c40,c155,c180,c180,c181,c17,c4,c1153,...","w305,w13549,w22752,w11,w7225,w2565,w1106,w16,w...","c335,c101,c611,c189,c97,c144,c147,c101,c15,c76...","w231,w54,w1681,w54,w11506,w5714,w7,w54,w744,w1..."
1,2887834264226772863,"c44,c110,c101,c286,c106,c150,c101,c892,c632,c1...","w377,w54,w285,w57,w349,w54,w108215,w6,w47986,w...","c1265,c518,c74,c131,c274,c57,c768,c769,c368,c3...","w12508,w1380,w72,w27045,w276,w111"
2,-2687466858632038806,"c15,c768,c769,c1363,c650,c1218,c2361,c11,c90,c...","w875,w15450,w42394,w15863,w6,w95421,w25,w803,w...","c693,c100,c279,c99,c189,c532,c101,c189,c145,c1...","w140340,w54,w48398,w54,w140341,w54,w12856,w54,..."
3,-5698296155734268,"c473,c1528,c528,c428,c295,c15,c101,c188,c146,c...","w8646,w2744,w1462,w9,w54,w138,w54,w50,w110,w14...",NaN,NaN
4,-6719100304248915192,"c190,c147,c105,c219,c220,c101,c647,c219,c220,c...","w380,w54,w674,w133,w54,w134,w614,w54,w929,w307...","c644,c1212,c253,c199,c431,c452,c424,c207,c2,c1...","w4821,w1301,w16003,w928,w1961,w2565,w50803,w11..."


### 两种转list的方法，速度差的不是一点儿半点儿

In [9]:
start_time = time.time()

df_questions['wt_list'] = df_questions.wt.apply(split)

print('time cost:', time.time() - start_time)

start_time = time.time()

ddf_questions['wt_list'] = ddf_questions.wt.apply(split)

print('time cost:', time.time() - start_time)

time cost: 0.829592227935791
time cost: 0.007982015609741211


C:\Users\liuye\Anaconda3\lib\site-packages\dask\dataframe\core.py:1600: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  warnings.warn(msg)


In [10]:
start_time = time.time()

df_questions['wd_list'] = df_questions.wd.apply(split)

print('time cost:', time.time() - start_time)

start_time = time.time()

ddf_questions['wd_list'] = ddf_questions.wd.apply(split)

print('time cost:', time.time() - start_time)

time cost: 2.2994484901428223
time cost: 0.003500223159790039


C:\Users\liuye\Anaconda3\lib\site-packages\dask\dataframe\core.py:1600: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  warnings.warn(msg)


In [11]:
start_time = time.time()
df_questions['bag_of_words'] = df_questions.apply(lambda x : x['wt_list'] + x['wd_list'], axis = 1)
print('time cost:', time.time() - start_time)

time cost: 9.885101795196533


In [12]:
start_time = time.time()
ddf_questions['bag_of_words'] = ddf_questions.apply(lambda x : x['wt_list'] + x['wd_list'], axis = 1)
print('time cost:', time.time() - start_time)

time cost: 0.003995656967163086


C:\Users\liuye\Anaconda3\lib\site-packages\dask\dataframe\core.py:2207: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  warnings.warn(msg)


In [13]:
ddf_questions.head(5)

,question_id,ct,wt,cd,wd,wt_list,wd_list,bag_of_words
0,6555699376639805223,"c324,c39,c40,c155,c180,c180,c181,c17,c4,c1153,...","w305,w13549,w22752,w11,w7225,w2565,w1106,w16,w...","c335,c101,c611,c189,c97,c144,c147,c101,c15,c76...","w231,w54,w1681,w54,w11506,w5714,w7,w54,w744,w1...","[w305, w13549, w22752, w11, w7225, w2565, w110...","[w231, w54, w1681, w54, w11506, w5714, w7, w54...","[w305, w13549, w22752, w11, w7225, w2565, w110..."
1,2887834264226772863,"c44,c110,c101,c286,c106,c150,c101,c892,c632,c1...","w377,w54,w285,w57,w349,w54,w108215,w6,w47986,w...","c1265,c518,c74,c131,c274,c57,c768,c769,c368,c3...","w12508,w1380,w72,w27045,w276,w111","[w377, w54, w285, w57, w349, w54, w108215, w6,...","[w12508, w1380, w72, w27045, w276, w111]","[w377, w54, w285, w57, w349, w54, w108215, w6,..."
2,-2687466858632038806,"c15,c768,c769,c1363,c650,c1218,c2361,c11,c90,c...","w875,w15450,w42394,w15863,w6,w95421,w25,w803,w...","c693,c100,c279,c99,c189,c532,c101,c189,c145,c1...","w140340,w54,w48398,w54,w140341,w54,w12856,w54,...","[w875, w15450, w42394, w15863, w6, w95421, w25...","[w140340, w54, w48398, w54, w140341, w54, w128...","[w875, w15450, w42394, w15863, w6, w95421, w25..."
3,-5698296155734268,"c473,c1528,c528,c428,c295,c15,c101,c188,c146,c...","w8646,w2744,w1462,w9,w54,w138,w54,w50,w110,w14...",NaN,NaN,"[w8646, w2744, w1462, w9, w54, w138, w54, w50,...",[],"[w8646, w2744, w1462, w9, w54, w138, w54, w50,..."
4,-6719100304248915192,"c190,c147,c105,c219,c220,c101,c647,c219,c220,c...","w380,w54,w674,w133,w54,w134,w614,w54,w929,w307...","c644,c1212,c253,c199,c431,c452,c424,c207,c2,c1...","w4821,w1301,w16003,w928,w1961,w2565,w50803,w11...","[w380, w54, w674, w133, w54, w134, w614, w54, ...","[w4821, w1301, w16003, w928, w1961, w2565, w50...","[w380, w54, w674, w133, w54, w134, w614, w54, ..."


In [14]:
start_time = time.time()
df=df_questions[['question_id', 'bag_of_words']]
df.to_pickle(data_path+'question_words_bag.pickle')
print('time cost:', time.time() - start_time)

time cost: 9.929306507110596


### Remove all dataframe 

In [15]:
start_time = time.time()
del df
del df_questions
gc.collect()
print('time cost:', time.time() - start_time)

time cost: 1.696505069732666


### Reload data

Can we load the data to dask directly?

In [16]:
start_time = time.time()
df_bag=pd.read_pickle(data_path+'question_words_bag.pickle')
print('time cost:', time.time() - start_time)
print('End time:', time.time())

time cost: 3.0525426864624023
End time: 1498001447.051806


In [17]:
df_bag.head(5)

,question_id,bag_of_words
0,6555699376639805223,"[w305, w13549, w22752, w11, w7225, w2565, w110..."
1,2887834264226772863,"[w377, w54, w285, w57, w349, w54, w108215, w6,..."
2,-2687466858632038806,"[w875, w15450, w42394, w15863, w6, w95421, w25..."
3,-5698296155734268,"[w8646, w2744, w1462, w9, w54, w138, w54, w50,..."
4,-6719100304248915192,"[w380, w54, w674, w133, w54, w134, w614, w54, ..."


下面这一步很不可思议，只用了5秒多就统计好了全部数据

In [18]:
start_time = time.time()
df_bag['wt_counter'] = df_bag.bag_of_words.apply(Counter)
print('time cost:', time.time() - start_time)

df_bag.head(5)

time cost: 5.239213228225708


,question_id,bag_of_words,wt_counter
0,6555699376639805223,"[w305, w13549, w22752, w11, w7225, w2565, w110...","{'w22752': 1, 'w109': 1, 'w1106': 1, 'w1110': ..."
1,2887834264226772863,"[w377, w54, w285, w57, w349, w54, w108215, w6,...","{'w27045': 1, 'w21790': 1, 'w285': 1, 'w377': ..."
2,-2687466858632038806,"[w875, w15450, w42394, w15863, w6, w95421, w25...","{'w7490': 1, 'w241': 1, 'w42394': 1, 'w5481': ..."
3,-5698296155734268,"[w8646, w2744, w1462, w9, w54, w138, w54, w50,...","{'w112': 1, 'w50': 1, 'w2744': 1, 'w359': 1, '..."
4,-6719100304248915192,"[w380, w54, w674, w133, w54, w134, w614, w54, ...","{'w1301': 1, 'w109': 2, 'w241': 1, 'w674': 1, ..."


In [19]:
df_bag.wt_counter.loc[0]

Counter({'w1019': 1,
         'w1042': 1,
         'w109': 1,
         'w11': 2,
         'w1106': 1,
         'w111': 3,
         'w1110': 1,
         'w11506': 1,
         'w1166': 1,
         'w13549': 1,
         'w16': 1,
         'w1681': 1,
         'w22752': 1,
         'w23': 1,
         'w231': 1,
         'w25': 1,
         'w2565': 1,
         'w26377': 1,
         'w305': 1,
         'w31389': 1,
         'w3332': 1,
         'w37031': 1,
         'w54': 6,
         'w5714': 1,
         'w6': 2,
         'w69288': 1,
         'w7': 1,
         'w7225': 1,
         'w744': 2,
         'w7734': 1,
         'w9': 1})

## Count Occurancy of a word that occurs in a question, including description and title

In [20]:
def CountWords(row):
    for w in row:
        if w not in word_dict.keys():
            word_dict[w] = 1
        else:
            word_dict[w] += 1
    return
word_dict = {}
#word_dict = dict.fromset
start_time = time.time()
_ = df_bag.wt_counter.apply(CountWords)
print('time cost:', time.time() - start_time)

print(len(word_dict))

time cost: 4.929028749465942
248913


## 计算逆文本频率指数 IDF

$$ IDF = log(\frac{D}{D_w}) $$

D： 所有的Question的总数

Dw：词 w 出现在Dw 篇文章中

比如 的 几乎出现在所有的问题中，其IDF 就几乎为零。

秒执行

In [21]:
idf_dict={}
D = len(df_bag)
for k,v in word_dict.items():
    idf_dict[k] = np.log2(float(D)/v)

In [22]:
idf_dict

{'w11443': 18.416995396494414,
 'w433929': 18.416995396494414,
 'w399330': 17.416995396494414,
 'w137190': 16.83203289577326,
 'w459565': 18.416995396494414,
 'w89696': 17.416995396494414,
 'w269235': 17.416995396494414,
 'w16163': 14.83203289577326,
 'w206448': 17.416995396494414,
 'w126892': 16.095067301607052,
 'w244275': 15.83203289577326,
 'w183166': 14.095067301607054,
 'w135366': 18.416995396494414,
 'w62629': 13.609640474436812,
 'w129947': 16.095067301607052,
 'w57704': 13.95756377785712,
 'w169883': 18.416995396494414,
 'w298010': 17.416995396494414,
 'w67865': 15.416995396494416,
 'w389978': 18.416995396494414,
 'w351536': 18.416995396494414,
 'w423156': 18.416995396494414,
 'w2137': 8.5793674633230115,
 'w263196': 17.416995396494414,
 'w18070': 15.83203289577326,
 'w31144': 16.83203289577326,
 'w105458': 12.662107894330948,
 'w283515': 17.416995396494414,
 'w284174': 18.416995396494414,
 'w23964': 12.893433440437404,
 'w383884': 18.416995396494414,
 'w206038': 18.4169953964

# 根据 TF-IDF 计算 Question与Topic的相关性

$$ TF-IDF = TF_1\cdot IDF_1 + TF_2 \cdot IDF_2 + ... + TF_N \cdot IDF_N $$

TF1: 词1在此Question 出现的频率 $$ TFx = \frac{词_x在此question中出现的次数}{此Question中的总词数}$$ 

此处的词指来自Topic中的词。如Topic为w32,w1234 则计算每一篇文章与W32, w1234的相关性。



用的时候再加载，减少不必要的内存占用

In [23]:
df_topics = pd.read_csv(data_path+'topic_info.txt', header=None, names=['topic_id', 'pid', 'cn', 'wn', 'cd', 'wd'],sep='\t')
df_question_topic = pd.read_csv(data_path+'question_topic_train_set.txt', header=None, names=['question_id', 'topic_id'],sep='\t')

In [24]:
df_topics[df_topics['topic_id'] == 738845194850773558].wd[0]

'w0,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11,w12,w13,w14,w15,w16,w17,w18,w15,w6,w19,w20,w21,w22,w23'

### 数据预处理

1. 在df_bag 中增加一列，包含每一个question的词的总数， 就是下面的 total_word
2. 按照上面处理 question的方法，处理topics, 也要有 wt_counter 与 total_word


#### Adding bag_of_words, wt_counter and total_words to df_topics

In [25]:
def topic_word_bag(row):
    if type(row.wn) == float and type(row.wd) == float:
        return []
    elif type(row.wn) == float:
        return row.wd.split(',')
    elif type(row.wd) == float:
        return row.wn.split(',')
    return (row.wn + ',' + row.wd).split(',') 

df_topics['bag_of_words'] = df_topics.apply(topic_word_bag, axis = 1)

In [26]:
def topic_wt_counter(row):
    d = dict()
    for word in row:
        if word not in d.keys():
            d[word] = 1
        else:
            d[word] += 1
    return d
df_topics['wt_counter'] = df_topics.bag_of_words.apply(topic_wt_counter)

In [27]:
def total_word(row):
    return sum(row.values())
df_topics['total_word'] = df_topics.wt_counter.apply(total_word)

#### Adding total_words to df_bag

In [28]:
df_bag['total_word'] = df_bag.wt_counter.apply(total_word)

In [29]:
def tf_idf(topic_id, question_id):
    index1 = df_topics[df_topics['topic_id'] == topic_id].index[0]
    topic_word = df_topics[df_topics['topic_id'] == topic_id].bag_of_words[index1]

    index2 = df_bag[df_bag['question_id'] == question_id].index[0]
    word_dict = df_bag[df_bag['question_id'] == question_id].wt_counter[index2]

    total_word = df_bag[df_bag['question_id'] == question_id].total_word[index2]

    
    tf_idf_value = 0
    for word in topic_word:
        if word in word_dict:
            tf_idf_value += idf_dict[word]*word_dict[word]/total_word
    return tf_idf_value

In [30]:
tf_idf(-3149765934180654494, 2887834264226772863)

0.019724183082528223

In [31]:
def split_to_list(row):
    return row.split(',')
df_question_topic['topic_id_list'] = df_question_topic.topic_id.apply(split_to_list)

In [32]:
df_question_topic['topic_count'] = df_question_topic.topic_id_list.apply(len)

In [33]:
sub_df = df_question_topic[df_question_topic['topic_count'] == 1]

In [34]:
sub_df.loc[1][1]

'-3149765934180654494'

In [35]:
def tf_idf2(ser):
    return tf_idf(int(ser.topic_id_list[0]), ser.question_id)

In [36]:
sub_df.head(5)

,question_id,topic_id,topic_id_list,topic_count
1,2887834264226772863,-3149765934180654494,[-3149765934180654494],1
2,-2687466858632038806,-760432988437306018,[-760432988437306018],1
9,3174606710238304130,-4115748438709160582,[-4115748438709160582],1
18,-3679570071031716995,-9176307901497282391,[-9176307901497282391],1
21,6004514913022607006,-8966465280115387956,[-8966465280115387956],1


In [37]:
df_topics[df_topics['topic_id'] == -3149765934180654494]

,topic_id,pid,cn,wn,cd,wd,bag_of_words,wt_counter,total_word
769,-3149765934180654494,-5388067101870430963,"c1487,c378",w17245,"c70,c110,c1216,c1303,c11,c2308,c1201,c212,c378","w10730,w2741,w6,w17246,w17247","[w17245, w10730, w2741, w6, w17246, w17247]","{'w17245': 1, 'w6': 1, 'w17246': 1, 'w10730': ...",6


In [38]:
df_topics[ df_topics['topic_id'] == 738845194850773558]

,topic_id,pid,cn,wn,cd,wd,bag_of_words,wt_counter,total_word
0,738845194850773558,-5833678375673307423,"c0,c1",w0,"c0,c1,c2,c3,c4,c5,c6,c7,c0,c1,c8,c9,c10,c11,c1...","w0,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11,w12,w13,...","[w0, w0, w1, w2, w3, w4, w5, w6, w7, w8, w9, w...","{'w10': 1, 'w18': 1, 'w22': 1, 'w5': 1, 'w2': ...",27


In [39]:
print('df_topics[df_topics[\'topic_id\'] == 738845194850773558].wn=', df_topics[df_topics['topic_id'] == 738845194850773558].wn)
print('df_topics[df_topics[\'topic_id\'] == 738845194850773558].wn[0]=', df_topics[df_topics['topic_id'] == 738845194850773558].wn[0])
print()

print('df_topics[df_topics[\'topic_id\'] == -3149765934180654494].wn=', df_topics[df_topics['topic_id'] == -3149765934180654494].wn)
print('df_topics[df_topics[\'topic_id\'] == -3149765934180654494].wn[0]=', df_topics[df_topics['topic_id'] == -3149765934180654494].wn[769])

# why? !!!

df_topics[df_topics['topic_id'] == 738845194850773558].wn= 0    w0
Name: wn, dtype: object
df_topics[df_topics['topic_id'] == 738845194850773558].wn[0]= w0

df_topics[df_topics['topic_id'] == -3149765934180654494].wn= 769    w17245
Name: wn, dtype: object
df_topics[df_topics['topic_id'] == -3149765934180654494].wn[0]= w17245


In [40]:
df_topics[df_topics['topic_id'] == -3149765934180654494].index[0]

769

In [41]:
#tf_idf2(sub_df.loc[1])
print(int(sub_df.loc[1].topic_id_list[0]),sub_df.loc[1].question_id)
tf_idf(-3149765934180654494 ,2887834264226772863)

-3149765934180654494 2887834264226772863


0.019724183082528223

In [42]:
start_time = time.time()
a = sub_df.head(10000).apply(tf_idf2, axis = 1)
print('time cost:', time.time() - start_time)

time cost: 27.73903465270996


In [43]:
sub_df.head(114900)

,question_id,topic_id,topic_id_list,topic_count
1,2887834264226772863,-3149765934180654494,[-3149765934180654494],1
2,-2687466858632038806,-760432988437306018,[-760432988437306018],1
9,3174606710238304130,-4115748438709160582,[-4115748438709160582],1
18,-3679570071031716995,-9176307901497282391,[-9176307901497282391],1
21,6004514913022607006,-8966465280115387956,[-8966465280115387956],1
23,-6317692530143404667,-4175044003751472418,[-4175044003751472418],1
26,-2335474937104234672,6493581724141198741,[6493581724141198741],1
28,-6473476649740451635,3247902953025099742,[3247902953025099742],1
31,554647747767379078,2460528390051567153,[2460528390051567153],1
32,7964675238481673415,-2288652390863265229,[-2288652390863265229],1


In [44]:
a.sort()

C:\Users\liuye\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort is deprecated, use sort_values(inplace=True) for INPLACE sorting
  if __name__ == '__main__':


In [45]:
a

39944     0.000000
24534     0.000000
24513     0.000000
24512     0.000000
8826      0.000000
24442     0.000000
8862      0.000000
24441     0.000000
24420     0.000000
8887      0.000000
24400     0.000000
24393     0.000000
24376     0.000000
24367     0.000000
24337     0.000000
8928      0.000000
24336     0.000000
24334     0.000000
24313     0.000000
24307     0.000000
24559     0.000000
24295     0.000000
24571     0.000000
8810      0.000000
24827     0.000000
24814     0.000000
24804     0.000000
24787     0.000000
24784     0.000000
24783     0.000000
           ...    
39040    10.765944
38223    10.772014
11023    10.973747
36711    11.034490
1449     11.322911
17618    11.422856
5016     11.472346
29837    11.522178
21345    11.530810
35650    12.017389
10802    12.464419
4793     12.536169
19944    12.624126
27867    13.152722
31887    13.587493
14089    14.423635
4237     15.148951
26163    15.205886
3979     15.434888
34200    15.492865
16656    15.698864
2310     15.

In [ ]:
sub_df['tf_idf'] = sub_df.apply(tf_idf2, axis = 1)

## 考查TF_IDF的分布情况

### 计算单topic的Question的TF-IDF分布情况

### 计算多topic的Question的TF-IDF分布情况

In [ ]:
# your code here ...

## 分析多Topic时，Topic的位置与TF-IDF的关系

In [ ]:
# your code here ...

## 研究 Topic 继承关系对Topic赋值的影响

## 研究同义词对 Topic赋值的影响